# СОДЕРЖАНИЕ НОУТБУКА

В данном ноутбуке проведены исследования по 3-4 шагам из README:

- === ШАГ 3: Трансляция ===
- === ШАГ 4: Моделирование ===

# === ШАГ 3: Трансляция ===

```
Для решения нашей задачи был выбран следующий подход и метрики:

Оталкиваясь от исследований в этой области было решено строить алгоритм рекомендательной системы банковских продуктов
на основе гибридного подхода, которая сочетает метод коллаборативной фильтрации (ALS) и демографический подход.

Этапы построения алгоритма РС:

   I. Построение ALS-алгоритма: 

    - Построение UI-матрицы «клиенты-продукты» для каждого клиента на основе неявной оценки продукта по определенной шкале в зависимости
от частоты использования этого банковского продукта; 
    - Прогнозирование оценки еще не оцененных продуктов на основе item-based;

   II. Построение алгоритма по демографическому подходу: 

    - Группировка пользователей в кластеры на основе демографических
данных, такие как (возраст, пол, доход, активность, стаж и место жительства) с применением алгоритма k-средних;
    - Построение матрицы «кластеры-продукты», содержащей
среднее значение оценок, присвоенных продукту всеми клиентами в кластере методом динамического взвешивания.

   III. Объединение базовых алгоритмов: 

    - Получение прогнозов по обоим алгоритмам, объединение и сортировка рейтингов в порядке убывания и рекомендация клиенту первых N продуктов.

Разбиение данных:

 - Данные разделяются на обучающий и тестовый набор по глобальному разделению по времени (Global Time Split, Temporal Global). Выбирается одна точка во времени ts: все события до неё входят в тренировочную выборку (прошлое), после неё — в тестовую (будущее). 
 - На основе обучающего набора формируется UI-матрица и матрица «продукты—кластеры».
 - Для формирования тестового набора выбираются клиенты как минимум с тремя продуктами в наборе данных с оценками продуктов.

Метрики:

Выбрана метрика ранжирования NDCG@5, поскольку мы решаем задачу ранжирования и используем алгоритмом ALS. Она принимает значение от 0 (предлагаемый порядок никак не соответствует истинному) до 1 (предлагаемый порядок в точности соответствует истинному). 
```

# === ШАГ 4: Моделирование ===

### 1. Загрузка библиотек

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

# Будем сохранять построенные графики в папку "assets"
ASSETS_DIR = "assets"
if not os.path.exists(ASSETS_DIR):
    os.mkdir(ASSETS_DIR)

KeyboardInterrupt: 

### 2. Загрузка очищенных данных
- train_ver2_clean.parquet
- products_catalog.parquet

In [2]:
# Загрузка данных

df = pd.read_parquet("data/train_ver2_clean.parquet")
products_catalog = pd.read_parquet("data/products_catalog.parquet")

### 3. ALS-модель. Таргет

Для заполнения UI-матрицы необходимо посчитать число неявных оценок клиентов путем группировки и агрегирования транзакционных данных.

In [3]:
# В качестве значений для заполнения UI-матрицы будем использовать отношение длительности владения продуктом к стажу на текущий месяц.

# Посчитаем кол-во месяцев владения клиентом продуктом, т.е. посчитаем кумулятивную сумму, при этом обнуляя счетчик, если продукт был удален.
# Затем нормируем это значение в каждом месяце на текущий стаж клиента.
# Полученное значение будет использовано для заполнения UI-матрицы.
# Его макс значение 1 будет означать максимальную релевантность продукта клиенту, а 0 - нерелевантность.

df = df.sort_values(by=["ncodpers", "fecha_dato"]).reset_index(drop=True)

for colname in products_catalog["names"].to_list():

    df[colname + "_cumsum"] = df.groupby(["ncodpers", df[colname].eq(0).cumsum()])[
        colname
    ].cumsum()
    df[colname + "_cumsum_rank"] = df.groupby("ncodpers")["fecha_dato"].rank()

    df[colname + "_cumsum_ratio"] = round(
        df[colname + "_cumsum"] / df[colname + "_cumsum_rank"],
        2,
    )

In [ ]:
# Проверка результатов агрегирования для клиента с ncodpers = 1108076 по продукту "Текущие счета"

ncodpers = 1108076
colname = "ind_cco_fin_ult1"
display(
    df[df["ncodpers"] == ncodpers][
        [
            "fecha_dato",
            "ncodpers",
            colname,
            colname + "_cumsum",
            colname + "_cumsum_rank",
            colname + "_cumsum_ratio",
        ]
    ].sort_values(by="fecha_dato")
)

In [5]:
# Удаляем лишние столбцы

cumsum_cols = [i + "_cumsum" for i in products_catalog["names"].to_list()]
df = df.drop(columns=cumsum_cols)

cumsum_rank_cols = [i + "_cumsum_rank" for i in products_catalog["names"].to_list()]
df = df.drop(columns=cumsum_rank_cols)

### 4. ALS-модель. Разбиение данных

Разбиваем данные на тренировочную, тестовую выборки по глобальному разделению по времени (Global Time Split, Temporal Global). 

Выбирается одна точка во времени ts: все события до неё входят в тренировочную выборку (прошлое), после неё — в тестовую (будущее).

In [ ]:
# Зададим точку разбиения так, чтобы в тест попали данные только за последний месяц

train_test_global_time_split_date = pd.to_datetime("2016-05-28").date()

train_test_global_time_split_idx = df["fecha_dato"].astype("date32[pyarrow]") < train_test_global_time_split_date

events_train = df[train_test_global_time_split_idx].reset_index(drop=True)
print(f"Уникальных пользователей в train: {len(events_train["ncodpers"].drop_duplicates(keep="last"))}")
events_test = df[~train_test_global_time_split_idx].reset_index(drop=True)
print(f"Уникальных пользователей в test: {len(events_test["ncodpers"].drop_duplicates(keep="last"))}")

print(f"Количество взаимодействий в train: {len(events_train)}")
print(f"Количество взаимодействий в test: {len(events_test)}")

In [ ]:
# Отделим транзакционную часть от персональной

cumsum_ratio_cols = [i + "_cumsum_ratio" for i in products_catalog["names"].to_list()]

events_train_trans = events_train[
    sum(
        [
            ["fecha_dato"],
            ["ncodpers"],
            ["prod_count"],
            products_catalog["names"].to_list(),
            cumsum_ratio_cols,
        ],
        [],
    )
]
events_test_trans = events_test[
    sum(
        [
            ["fecha_dato"],
            ["ncodpers"],
            ["prod_count"],
            products_catalog["names"].to_list(),
            cumsum_ratio_cols,
        ],
        [],
    )
]

events_train_trans

In [ ]:
# Визуализируем для наглядности среднюю динамику по *_cumsum_ratio

cumsum_ratio_cols_df = (
    events_train_trans.groupby("fecha_dato")[cumsum_ratio_cols].mean().reset_index()
)

axs = plt.figure(figsize=(36, 26))

for i in cumsum_ratio_cols:
    plt.plot(cumsum_ratio_cols_df["fecha_dato"], cumsum_ratio_cols_df[i])

plt.legend(products_catalog["describe"], loc=2, prop={"size": 20})
plt.tick_params(axis="x", rotation=45, labelsize=20)
plt.tick_params(axis="y", labelsize=20)
plt.title("Динамика средней доли стажа по продукту", fontsize=20)
plt.ylabel("Средняя доля стажа по продукту", fontsize=20)
plt.grid()
plt.savefig(os.path.join(ASSETS_DIR, "6_cumsum_ratio_dynamic.png"))

In [ ]:
# Удалим лишние данные из наборов

# В трейне оставляем данные только последнего месяца
events_train_trans = events_train_trans.drop_duplicates(
    subset="ncodpers", keep="last"
).reset_index()

# В трейне оставляем только тех клиентов, у которых есть хотя бы 1 продукт
events_train_trans = events_train_trans[events_train_trans["prod_count"] >= 1].reset_index()
print(
    f"Уникальных пользователей в train c 1 продуктом и более: {len(events_train_trans)}"
)

# На тесте оставляем только тех клиентов, у которых есть хотя бы 3 продукта
events_test_trans=events_test_trans[events_test_trans["prod_count"]>=3].reset_index()
print(f"Уникальных пользователей в test c 3 продуктами и более: {len(events_test_trans["ncodpers"].drop_duplicates(keep="last"))}")

# На трейне оставляем только идентификаторы клиентов и целевые колонки по продуктам *_cumsum_ratio
events_train_trans = events_train_trans[sum([["ncodpers"], cumsum_ratio_cols], [])]

# Переименуем *_cumsum_ratio в трейне в исходные названия
d={}
prod_names = products_catalog["names"].to_list()
for i,j in enumerate(cumsum_ratio_cols):
    d[j] = prod_names[i]
events_train_trans.rename(columns=d, inplace=True)

# На тесте оставляем только идентификаторы клиентов и целевые колонки по продуктам *_cumsum_ratio
events_test_trans = events_test_trans[sum([["ncodpers"], cumsum_ratio_cols], [])]

# Переименуем *_cumsum_ratio в тесте в исходные названия
events_test_trans.rename(columns=d, inplace=True)

# На тесте оставим только тех пользователей, которые есть в трейне
events_test_trans = events_test_trans[
    events_test_trans["ncodpers"].isin(events_train_trans["ncodpers"])
].reset_index(drop=True)

In [10]:
# Сохраним на локал промежуточные данные

events_train_trans.to_parquet("events_train_trans.parquet")
events_test_trans.to_parquet("events_test_trans.parquet")

### 5. ALS-модель. Обучение

Перед выполнением кода ниже можно почистить kernel

In [ ]:
# Импортируем необходимые библиотеки

import numpy as np
import pandas as pd
import os
import sklearn.preprocessing
import scipy
import sys
from implicit.als import AlternatingLeastSquares
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

# Будем сохранять построенные графики в папку "assets"
ASSETS_DIR = "assets"
if not os.path.exists(ASSETS_DIR):
    os.mkdir(ASSETS_DIR)

In [2]:
# Загружаем данные

products_catalog = pd.read_parquet("data/products_catalog.parquet")

events_train_trans = pd.read_parquet("events_train_trans.parquet")
events_test_trans = pd.read_parquet("events_test_trans.parquet")

In [ ]:
# Список пользователей из трейна и теста

user_list = list(
    set(
        events_train_trans["ncodpers"].to_list()
        + events_test_trans["ncodpers"].to_list()
    )
)
len(user_list)

In [4]:
# Перекодируем идентификаторы клиентов из имеющихся в последовательность 0, 1, 2, ...

user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(user_list)

events_train_trans["ncodpers_enc"] = user_encoder.transform(
    events_train_trans["ncodpers"]
)

events_test_trans["ncodpers_enc"] = user_encoder.transform(
    events_test_trans["ncodpers"]
)

# В качестве идентификаторов продуктов возьмем индексы из таблицы products_catalog
products_catalog = products_catalog.reset_index().rename(columns={"index": "names_enc"})

In [ ]:
# Делаем в качестве индексов - кодированные идентификаторы клиентов, в качестве названий колонок - кодированные идентификаторы продуктов

events_train_trans_matrix = events_train_trans.drop(columns="ncodpers").set_index(
    "ncodpers_enc"
)
events_train_trans_matrix.columns = products_catalog["names_enc"]

events_train_trans_matrix

In [ ]:
# Постром UI-матрицу (sparse-матрицу формата CSR)

user_item_matrix_train = scipy.sparse.csr_matrix(events_train_trans_matrix.values)
print(
    f"Cтепень разреженности UI-матрицы: {(events_train_trans_matrix == 0).sum(axis=1).sum()/(events_train_trans_matrix.shape[0] * events_train_trans_matrix.shape[1])}"
)
print(
    f"Размеры памяти, требуемой для хранения UI-матрицы: {sum([sys.getsizeof(i) for i in user_item_matrix_train.data])/1024**3}"
)

In [ ]:
# Обучим ALS-модель. Для бейзлайна возьмём количество латентных факторов для матриц $P, Q$, равным 50.

als_model = AlternatingLeastSquares(
    factors=50, iterations=50, regularization=0.05, random_state=0
)
als_model.fit(user_item_matrix_train)

In [8]:
# Чтобы получить рекомендации для пользователя с помощью модели ALS, используем такую функцию:


def get_recommendations_als(
    user_item_matrix,
    model,
    user_id,
    user_encoder,
    products_catalog,
    include_seen=True,
    n=5,
):
    """
    Возвращает отранжированные рекомендации для заданного клиента
    """
    user_id_enc = user_encoder.transform([user_id])[0]
    recommendations = model.recommend(
        user_id_enc,
        user_item_matrix[user_id_enc],
        filter_already_liked_items=not include_seen,
        N=n,
    )
    recommendations = pd.DataFrame(
        {"names_enc": recommendations[0], "score": recommendations[1]}
    )
    recommendations = recommendations.merge(
        products_catalog, on="names_enc", how="left"
    )

    return recommendations

In [ ]:
# Пример рекомендации для клиента 270434

ncodpers = 270434

recs = get_recommendations_als(
    user_item_matrix_train,
    als_model,
    ncodpers,
    user_encoder,
    products_catalog,
    include_seen=True,
    n=24,
)

recs.head()

In [ ]:
# Проверим статус продуктов клиента в трейне и тесте и сравним с рекомендациями

recs.merge(
    events_train_trans_matrix.loc[user_encoder.transform([ncodpers])[0]], on="names_enc"
).merge(
    events_test_trans[events_test_trans["ncodpers"] == ncodpers][
        products_catalog["names"].to_list()
    ].T.reset_index(),
    left_on="names",
    right_on="index",
    how="left",
).drop(
    columns="index"
).set_axis(
    ["names_enc", "als_score", "names", "describe", "train_status", "test_status"],
    axis=1,
).head()

# Т.к. был поставлен флажок include_seen=True, то рекомендации выдаются те, которые были у клиента и в трейне и тесте. Все корректно.

In [11]:
# Составим список всех возможных ncodpers (перекодированных)
user_list_encoded = range(len(user_encoder.classes_))

# Получим по 3 рекомендации для всех клиентов, исключая оформленные продукты
als_recommendations = als_model.recommend(
    user_list_encoded,
    user_item_matrix_train[user_list_encoded],
    filter_already_liked_items=True,
    N=3,
)

In [ ]:
# Преобразуем полученные рекомендации в табличный формат

names_enc = als_recommendations[0]
als_scores = als_recommendations[1]

als_recommendations = pd.DataFrame(
    {
        "ncodpers_enc": user_list_encoded,
        "names_enc": names_enc.tolist(),
        "score": als_scores.tolist(),
    }
)

als_recommendations = als_recommendations.explode(
    ["names_enc", "score"], ignore_index=True
)
als_recommendations

In [ ]:
# Приведем типы данных в исходные идентификаторы

als_recommendations["names_enc"] = als_recommendations["names_enc"].astype("int")
als_recommendations["score"] = als_recommendations["score"].astype("float")

# получаем изначальные идентификаторы
als_recommendations["ncodpers"] = user_encoder.inverse_transform(
    als_recommendations["ncodpers_enc"]
)
als_recommendations = als_recommendations.merge(
    products_catalog, on="names_enc", how="left"
)

als_recommendations = als_recommendations.drop(columns=["ncodpers_enc", "names_enc"])
als_recommendations

In [14]:
# Сохраним на локал промежуточные данные

als_recommendations.to_parquet("als_recommendations.parquet")
als_model.save("als_model")

events_train_trans.to_parquet("events_train_trans.parquet")
events_test_trans.to_parquet("events_test_trans.parquet")

### 6. ALS-модель. Метрика

Перед выполнением кода ниже можно почистить kernel

In [ ]:
# Импортируем необходимые библиотеки

import numpy as np
import pandas as pd
import os
import sklearn.preprocessing
import sklearn.metrics
import scipy
import sys
from implicit.als import AlternatingLeastSquares
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

# Будем сохранять построенные графики в папку "assets"
ASSETS_DIR = "assets"
if not os.path.exists(ASSETS_DIR):
    os.mkdir(ASSETS_DIR)

In [ ]:
# Загружаем данные

products_catalog = pd.read_parquet("data/products_catalog.parquet")

events_train_trans = pd.read_parquet("events_train_trans.parquet")
events_test_trans = pd.read_parquet("events_test_trans.parquet")

als_recommendations = pd.read_parquet("als_recommendations.parquet")

als_model = AlternatingLeastSquares(
    factors=50, iterations=50, regularization=0.05, random_state=0
)
als_model = als_model.load("als_model.npz")

In [ ]:
# Переструктурируем обучающий набор и оставим в нем только активные продукты клиента

events_train_trans["score"] = events_train_trans[
    products_catalog["names"].to_list()
].values.tolist()
events_train_trans["names"] = [products_catalog["names"].to_list()] * len(
    events_train_trans
)

events_train_trans = events_train_trans[["ncodpers", "names", "score"]]
events_train_trans = events_train_trans.explode(["score", "names"])
events_train_trans = events_train_trans[events_train_trans["score"] > 0].reset_index(
    drop=True
)
events_train_trans

In [ ]:
# Переструктурируем тестовый набор и оставим в нем только активные продукты клиента

events_test_trans["score"] = events_test_trans[
    products_catalog["names"].to_list()
].values.tolist()
events_test_trans["names"] = [products_catalog["names"].to_list()] * len(
    events_test_trans
)

events_test_trans = events_test_trans[["ncodpers", "names", "score"]]
events_test_trans = events_test_trans.explode(["score", "names"])
events_test_trans = events_test_trans[events_test_trans["score"] > 0].reset_index(
    drop=True
)
events_test_trans

In [ ]:
# Добавим в датафрейм с рекомендациями истинные оценки из тестовой выборки и обучающей выборки

als_recommendations = als_recommendations.merge(
    events_train_trans[["ncodpers", "names", "score"]].rename(
        columns={"score": "score_train"}
    ),
    on=["ncodpers", "names"],
    how="left",
)

als_recommendations = als_recommendations.merge(
    events_test_trans[["ncodpers", "names", "score"]].rename(
        columns={"score": "score_test"}
    ),
    on=["ncodpers", "names"],
    how="left",
)

als_recommendations

In [6]:
# Функция подсчета метрики NDCG для одного клиента с помощью sklearn.metrics.ndcg_score


def compute_ndcg(rating: pd.Series, score: pd.Series, k):
    if len(rating) < 2:
        return np.nan
    ndcg = sklearn.metrics.ndcg_score(
        np.asarray([rating.to_numpy()]), np.asarray([score.to_numpy()]), k=k
    )
    return ndcg

In [ ]:
# Посчитаем метрику NDCG для k=3 для всех пользователей из тестовой выборки.
# В результате каждому пользователю будет соответствовать одно значение NDCG@3

rating_test_idx = ~als_recommendations["score_test"].isnull()
ndcg_at_3_scores = (
    als_recommendations[rating_test_idx]
    .groupby("ncodpers")
    .apply(lambda x: compute_ndcg(x["score_test"], x["score"], k=3))
)
print(f"NDCG@3 mean: {ndcg_at_3_scores.mean()}")
print(
    f"len(NDCG@3) of notna values]: {len(ndcg_at_3_scores[~ndcg_at_3_scores.isna()])}"
)

In [ ]:
# Пример рекомендации для клиента 16056

als_recommendations[als_recommendations["ncodpers"] == 16056]

# Алгоритм выдал в качестве рекомендаций продукты (ind_nom_pens_ult1 и ind_nomina_ult1),
# которых не было в момент обучения (train_score), но они есть на тесте (test_score).
# Т.е. клиент оформил эти продукты в следующем месяце.

In [9]:
# Сохраним на локал промежуточные данные

als_recommendations.to_parquet("als_recommendations.parquet")

### 7. K-MEANS: Алгоритм рекомендаций по демографическому признаку. Загрузка данных

Перед выполнением кода ниже можно почистить kernel

In [1]:
# Импортируем необходимые библиотеки

import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import f1_score

In [2]:
# Загружаем данные

df = pd.read_parquet("data/train_ver2_clean.parquet")
products_catalog = pd.read_parquet("data/products_catalog.parquet")

# Уберем лишние колонки
df = (
    df.drop(columns=["fecha_dato", "prod_count", "ncodpers", "nomprov"])
    .drop_duplicates(keep="last")
    .reset_index(drop=True)
)

# Преобразуем данные
df["ind_actividad_cliente"] = df["ind_actividad_cliente"].apply(lambda x: int(x))
df["ind_nuevo"] = df["ind_nuevo"].apply(lambda x: int(x))
df["age"] = df["age"].apply(lambda x: int(x))
df["cod_prov"] = df["cod_prov"].apply(lambda x: int(x))

print("shape: ", df.shape)

shape:  (7639945, 34)


### 8. K-MEANS. Кодирование признаков

In [3]:
# Создаем трансформеры OneHotEncoder и StandardScaler и кодируем соответствующие колонки

cat_cols = [
    "sexo",
    "ind_nuevo",
    "indext",
    "canal_entrada",
    "cod_prov",
    "ind_actividad_cliente",
    "segmento",
]
num_cols = ["age", "antiguedad", "renta"]

one_hot_drop = OneHotEncoder(drop="if_binary", sparse_output=False)
one_hot_drop.fit(df[cat_cols])

standart_scaler = StandardScaler()
standart_scaler.fit(df[num_cols])


def coding_features(one_hot_drop, standart_scaler, df, cat_cols, num_cols):

    drop_res = one_hot_drop.transform(df[cat_cols])
    scaler_res = standart_scaler.transform(df[num_cols])

    drop_res = pd.DataFrame(drop_res, columns=one_hot_drop.get_feature_names_out())
    scaler_res = pd.DataFrame(scaler_res, columns=num_cols)

    return pd.concat([scaler_res, drop_res], axis=1)

### 9. K-MEANS. Разбиение данных

In [4]:
# Для экономии ресурсов, возьмем в качестве обучающей выборки первые 2 млн данных, а в качестве теста следущие 0.5 млн

train_size = 2000000
test_size = 500000

df_train = df[:train_size]
df_test = df[train_size : train_size + test_size]

del df

drop_res_train = coding_features(
    one_hot_drop, standart_scaler, df_train, cat_cols, num_cols
)
drop_res_train = pd.concat(
    [drop_res_train, df_train[products_catalog["names"].to_list()]], axis=1
)

print("train df shape: ", drop_res_train.shape)

train df shape:  (2000000, 249)


### 10. K-MEANS. Обучение

In [5]:
# Обучим модель кластеризации с дефолтными гиперпараметрами в качестве бейзлайна

kmeans = KMeans(n_clusters=8, random_state=0, n_init="auto").fit(
    drop_res_train.drop(columns=products_catalog["names"].to_list())
)
drop_res_train["labels"] = kmeans.predict(
    drop_res_train.drop(columns=products_catalog["names"].to_list())
)

In [6]:
# Данные из одного кластера

drop_res_train[drop_res_train["labels"] == 4]

,age,antiguedad,renta,sexo_V,ind_nuevo_1,indext_S,canal_entrada_004,canal_entrada_007,canal_entrada_013,canal_entrada_K00,...,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1,labels
10,-0.239512,-0.640471,0.069031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,4
15,0.097978,-0.640471,-1.127761,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,4
16,0.097978,-0.640471,-0.503957,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,4
21,0.266723,-0.640471,-0.340769,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,1,4
22,-0.492630,-0.640471,0.308638,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,1.279193,-0.809565,-0.113603,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,4
1999996,0.182351,-0.809565,-0.635164,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,4
1999997,0.688586,-0.809565,-0.650238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,1,4
1999998,-0.745747,-0.809565,-0.801114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,4


In [7]:
# Группируем по кластерам и осредняем продукты

drop_res_train_df = (
    drop_res_train.groupby("labels")[products_catalog["names"].to_list()]
    .mean()
    .reset_index()
)
drop_res_train_df

,labels,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,0,0.000000,0.000011,0.560477,0.000212,0.121295,0.001359,0.033591,0.009775,0.068340,...,0.000755,0.003575,0.003855,0.057238,0.053649,0.013598,0.001093,0.084003,0.086704,0.193559
1,1,0.000345,0.000026,0.519548,0.000866,0.178924,0.001203,0.000785,0.308116,0.089899,...,0.031212,0.030222,0.005667,0.124603,0.133994,0.068086,0.012337,0.135478,0.139683,0.256802
2,2,0.000000,0.000000,0.686553,0.000000,0.070831,0.004478,0.006020,0.009257,0.005037,...,0.000050,0.000373,0.000108,0.015148,0.014021,0.004298,0.000043,0.045774,0.047029,0.080841
3,3,0.000422,0.000171,0.549950,0.001247,0.187796,0.002082,0.006206,0.308022,0.118057,...,0.019620,0.033647,0.003159,0.150674,0.143341,0.074663,0.014233,0.143909,0.146499,0.271452
4,4,0.000000,0.000048,0.458799,0.000216,0.134934,0.004618,0.036149,0.010735,0.040740,...,0.000317,0.003702,0.000414,0.057790,0.054811,0.013789,0.001009,0.088994,0.091440,0.220143
5,5,0.000439,0.000122,0.537875,0.000928,0.156888,0.003193,0.016629,0.216397,0.103276,...,0.011284,0.028250,0.002255,0.151042,0.118834,0.070996,0.014028,0.112315,0.114763,0.249462
6,6,0.000000,0.000000,0.802662,0.000006,0.037674,0.000420,0.001708,0.002194,0.001418,...,0.000006,0.000080,0.000073,0.006251,0.004534,0.001127,0.000022,0.023482,0.024187,0.057703
7,7,0.000195,0.000014,0.526992,0.000945,0.111920,0.001377,0.000743,0.287165,0.070239,...,0.012397,0.013868,0.007917,0.089894,0.083988,0.040530,0.009514,0.080698,0.082514,0.170340


In [8]:
# Переструктурируем результаты в виде рекомендаций кластер-продукт

drop_res_train_df["score"] = drop_res_train_df[
    products_catalog["names"].to_list()
].values.tolist()
drop_res_train_df["names"] = [products_catalog["names"].to_list()] * len(
    drop_res_train_df
)
drop_res_train_df = drop_res_train_df[["labels", "names", "score"]]
drop_res_train_df = drop_res_train_df.explode(["score", "names"])
drop_res_train_df = drop_res_train_df[drop_res_train_df["score"] > 0].reset_index(
    drop=True
)

# Оставим только по 5 первых рекомендаций на каждый кластер

drop_res_train_df = (
    drop_res_train_df.sort_values(by=["labels", "score"], ascending=[True, False])
    .groupby("labels")
    .head(5)
    .reset_index(drop=True)
)
drop_res_train_df

# На первом месте в каждом кластере рекомендуется продукт "Текущие счета".
# Это будет своеобразная ТОП-рекомендация в том случае, если такого продукта у клиента нет, либо для холодных пользователей.

,labels,names,score
0,0,ind_cco_fin_ult1,0.560477
1,0,ind_recibo_ult1,0.193559
2,0,ind_cno_fin_ult1,0.121295
3,0,ind_ecue_fin_ult1,0.116771
4,0,ind_nom_pens_ult1,0.086704
5,1,ind_cco_fin_ult1,0.519548
6,1,ind_ctop_fin_ult1,0.308116
7,1,ind_recibo_ult1,0.256802
8,1,ind_cno_fin_ult1,0.178924
9,1,ind_ecue_fin_ult1,0.152536


In [9]:
# Получим кластеры для тестового набора

drop_res_test = coding_features(
    one_hot_drop, standart_scaler, df_test, cat_cols, num_cols
)
df_test["labels"] = kmeans.predict(drop_res_test)
df_test

,sexo,age,ind_nuevo,antiguedad,indext,canal_entrada,cod_prov,ind_actividad_cliente,renta,segmento,...,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1,labels
2000000,V,39,0,24,N,KFC,15,1,80634.87,02 - PARTICULARES,...,0,0,0,0,0,0,1,1,1,4
2000001,V,36,0,24,S,KFC,28,1,57818.46,02 - PARTICULARES,...,0,0,0,0,0,0,0,0,1,4
2000002,V,33,0,7,S,KHL,28,0,110591.88,02 - PARTICULARES,...,0,0,0,0,0,0,0,0,0,0
2000003,H,39,0,24,N,KFC,11,1,85903.44,02 - PARTICULARES,...,0,0,0,0,0,0,0,0,1,4
2000004,H,47,0,24,N,KAT,46,1,91032.78,02 - PARTICULARES,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2499995,H,53,0,138,N,KAT,28,0,342295.89,02 - PARTICULARES,...,0,0,0,0,0,0,0,0,0,5
2499996,V,40,0,173,N,KFC,28,1,350000.00,02 - PARTICULARES,...,0,0,0,0,0,0,0,0,0,5
2499997,V,49,0,173,N,KFC,17,1,105439.32,02 - PARTICULARES,...,0,0,0,0,0,0,0,0,0,7
2499998,V,38,0,173,N,KFC,8,0,91705.38,02 - PARTICULARES,...,0,0,0,0,0,0,0,0,0,7


In [10]:
# Переструктурируем тестовый набор по парам кластер-продукт, где test_score - индикатор существование продукта у клиента в тестовом наборе.

df_test["test_score"] = df_test[products_catalog["names"].to_list()].values.tolist()
df_test["names"] = [products_catalog["names"].to_list()] * len(df_test)
df_test = df_test[["labels", "names", "test_score"]]
df_test = df_test.explode(["test_score", "names"])
df_test = df_test[df_test["test_score"] > 0].reset_index(drop=True)
df_test

,labels,names,test_score
0,4,ind_cno_fin_ult1,1
1,4,ind_nomina_ult1,1
2,4,ind_nom_pens_ult1,1
3,4,ind_recibo_ult1,1
4,4,ind_cco_fin_ult1,1
...,...,...,...
538567,5,ind_cco_fin_ult1,1
538568,7,ind_cco_fin_ult1,1
538569,7,ind_ctop_fin_ult1,1
538570,7,ind_cco_fin_ult1,1


In [12]:
# Добавим к ним рекомендации и оставим в них только совпавшие с тестовыми парами кластер-продукт,
# где score - значения после кластеринга.

df_test_res = df_test.merge(drop_res_train_df, on=["labels", "names"], how="left")
df_test_res

,labels,names,test_score,score
0,4,ind_cno_fin_ult1,1,0.134934
1,4,ind_nomina_ult1,1,NaN
2,4,ind_nom_pens_ult1,1,0.09144
3,4,ind_recibo_ult1,1,0.220143
4,4,ind_cco_fin_ult1,1,0.458799
...,...,...,...,...
538567,5,ind_cco_fin_ult1,1,0.537875
538568,7,ind_cco_fin_ult1,1,0.526992
538569,7,ind_ctop_fin_ult1,1,0.287165
538570,7,ind_cco_fin_ult1,1,0.526992


In [20]:
# В качестве метрики в этой задаче, будем использовать F1-score.
# Скоры кластеринга будем заменять на 1 если score>0, и 0 - если score=NaN

df_test_res[df_test_res["score"].isna()]["score"] = 0
df_test_res[df_test_res["score"] > 0]["score"] = 1
f1_score(
    df_test_res["test_score"].to_list(),
    df_test_res["score"].to_list(),
    average="weighted",
)

# Получено высокое значение f1 = 0.95

/tmp/ipykernel_5426/891247080.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_res[df_test_res["score"]>0]["score"]=1


,labels,names,test_score,score
0,4,ind_cno_fin_ult1,1,1
1,4,ind_nomina_ult1,1,0
2,4,ind_nom_pens_ult1,1,1
3,4,ind_recibo_ult1,1,1
4,4,ind_cco_fin_ult1,1,1
...,...,...,...,...
538567,5,ind_cco_fin_ult1,1,1
538568,7,ind_cco_fin_ult1,1,1
538569,7,ind_ctop_fin_ult1,1,1
538570,7,ind_cco_fin_ult1,1,1
